In [ ]:
# config_path="../configs/query_encoder/config_PwC-Embedding_expr.json"
# data_schema="/workspace/configs/csv_schema/test_2.json"
# docs_jsonl_path="/workspace/results/searched_docs_db/search_documents_test.jsonl"
# auto_data_load=True


In [3]:
CONFIG_FILE = "../configs/query_encoder/config_gte-multilingual-base.json"

In [ ]:
from build_vectordb_search import build_vectordb_search

build_vectordb_search(
    config_path=CONFIG_FILE,
    data_schema="/workspace/configs/csv_schema/test_2.json",
    docs_jsonl_path="/workspace/data/expr/search_documents_20250912_013206.jsonl",
    auto_data_load=False,
)

/home/vscode/.local/lib/python3.11/site-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


Loading documents from /workspace/data/expr/search_documents_20250912_013206.jsonl: Auto Loading Data False
Loaded 2250 documents
모델 로딩: Alibaba-NLP/gte-multilingual-base (임베딩 차원을 1024으로 조절합니다)


A new version of the following files was downloaded from https://huggingface.co/Alibaba-NLP/new-impl:
- configuration.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/Alibaba-NLP/new-impl:
- modeling.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
Some weights of the model checkpoint at Alibaba-NLP/gte-multilingual-base were not used when initializing NewModel: ['classifier.bias', 'classifier.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of 

문서 인코딩 중...


Batches:   0%|          | 0/71 [00:00<?, ?it/s]

In [ ]:
import subprocess
import shlex

def run_command():
    """
    Constructs and executes the specified command using subprocess.
    """
    # --- Parameters ---
    script_name = "multi_hop_to_single_hop.py"
    input_file = "/workspace/data/rag_test_data/questions.jsonl"
    output_file = "/workspace/data/expr/singlehop_decompose.jsonl"
    question_field = "question"
    context_field = "context"
    mode = "decompose"
    model = "gemini-2.5-flash"

    # --- Command Construction ---
    # Using f-string to build the command string
    command_str = (
        f"python {script_name} "
        f"--input {shlex.quote(input_file)} "
        f"--output {shlex.quote(output_file)} "
        f"--question_field {shlex.quote(question_field)} "
        f"--context_field {shlex.quote(context_field)} "
        f"--mode {shlex.quote(mode)} "
        f"--model {shlex.quote(model)}"
    )

    print(f"🚀 Executing Command:\n{command_str}\n" + "-"*50)

    # --- Command Execution ---
    try:
        # Using shlex.split to handle arguments properly
        args = shlex.split(command_str)
        
        # Execute the command
        result = subprocess.run(
            args,
            check=True,        # Raise an exception if the command fails
            text=True,         # Capture stdout/stderr as text
            capture_output=True # Capture stdout and stderr
        )
        
        print("✅ Command Executed Successfully!")
        print("--- Standard Output (stdout) ---")
        print(result.stdout)
        if result.stderr:
            print("--- Standard Error (stderr) ---")
            print(result.stderr)

    except FileNotFoundError:
        print(f"❌ Error: The script '{script_name}' was not found.")
    except subprocess.CalledProcessError as e:
        print(f"❌ Command Failed with exit code {e.returncode}")
        print("--- Standard Output (stdout) ---")
        print(e.stdout)
        print("--- Standard Error (stderr) ---")
        print(e.stderr)
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

if __name__ == "__main__":
    run_command()

In [ ]:
from retrieval_system.main import main

In [3]:
import subprocess
import shlex

def run_bash_command():
    """
    주어진 Bash 명령어를 Python에서 실행하고 결과를 출력합니다.
    """
    # 실행할 Bash 명령어
    questions_path = "/workspace/data/expr/singlehop_decompose.jsonl"
    data_range = "1-50"
    top_k = 50
    device = "auto"
    schema_path = "/workspace/configs/csv_schema/test_3.json"

    # f-string을 사용하여 변수를 조합한 명령어 문자열 생성
    command_str = (
        f"python -m retrieval_system.main "
        f"--config_json {CONFIG_FILE} "
        f"--questions_jsonl {questions_path} "
        f"--range {data_range} "
        f"--top_k {top_k} "
        f"--device {device} "
        f"--schema_json {schema_path}"
    )

    # 명령어를 안전하게 리스트 형태로 분리 (shlex.split 사용 권장)
    command_list = shlex.split(command_str)
    
    print("🚀 실행할 명령어:")
    # 보기 좋게 출력하기 위해 각 인자를 줄바꿈하여 표시
    print(" \\\n    ".join(command_list))
    print("-" * 50)

    try:
        # subprocess.run을 사용하여 명령어 실행
        # check=True: 명령이 실패하면 CalledProcessError를 발생시킴
        # capture_output=True: stdout과 stderr를 캡처함
        # text=True: stdout과 stderr를 문자열로 디코딩함
        result = subprocess.run(
            command_list, 
            check=True, 
            capture_output=True, 
            text=True, 
            encoding='utf-8'
        )

        print("✅ 명령 실행 성공!")
        print("\n--- 표준 출력 (stdout) ---")
        print(result.stdout)
        
        # 표준 오류가 있다면 출력
        if result.stderr:
            print("\n--- 표준 오류 (stderr) ---")
            print(result.stderr)

    except FileNotFoundError:
        print(f"❌ 오류: '{command_list[0]}' 명령을 찾을 수 없습니다. 경로를 확인해주세요.")
        
    except subprocess.CalledProcessError as e:
        # 명령 실행 중 오류가 발생한 경우
        print(f"❌ 명령 실행 실패! (Return Code: {e.returncode})")
        print("\n--- 표준 출력 (stdout) ---")
        print(e.stdout)
        print("\n--- 표준 오류 (stderr) ---")
        print(e.stderr)

# 함수 실행
if __name__ == "__main__":
    run_bash_command()

🚀 실행할 명령어:
python \
    -m \
    retrieval_system.main \
    --config_json \
    ../configs/query_encoder/config_gte-multilingual-base.json \
    --questions_jsonl \
    /workspace/data/expr/singlehop_decompose.jsonl \
    --range \
    1-50 \
    --top_k \
    50 \
    --device \
    auto \
    --schema_json \
    /workspace/configs/csv_schema/test_3.json
--------------------------------------------------
✅ 명령 실행 성공!

--- 표준 출력 (stdout) ---
../results/vectordb/250912_062141/vector_db_gte_test_Alibaba-NLP_gte-multilingual-base.csv

--- Retrieval Complete ---
Output folder: ../results/retrieval_docs/250912_182734
Saved files:
- ../results/retrieval_docs/250912_182734/row_000001_Alibaba-NLP-gte-multilingual-base_345b050d.json
- ../results/retrieval_docs/250912_182734/row_000002_Alibaba-NLP-gte-multilingual-base_39e07752.json
- ../results/retrieval_docs/250912_182734/row_000003_Alibaba-NLP-gte-multilingual-base_39cf3a9e.json
- ../results/retrieval_docs/250912_182734/row_000004_Alibaba-NLP

In [ ]:
import subprocess

def run_preprocessing():
    """
    전처리 및 답변 생성 스크립트를 변수를 사용하여 실행합니다.
    """
    # ----------------------------------------------------
    # ✅ 각 인자를 변수로 분리하여 관리
    # ----------------------------------------------------
    input_dir = "/workspace/results/retrieval_docs/250912_182734/"
    max_rank = 5
    is_parallel = False  # 불리언(Boolean) 값으로 관리

    # ----------------------------------------------------
    # ✅ 리스트 형태로 명령어를 구성
    # ----------------------------------------------------
    command_list = [
        "python",
        "preprocess_and_generate_answer.py",
        "--input_dir", input_dir,
        "--max_rank", str(max_rank),        # 💡 숫자는 str()로 감싸 문자열로 변환합니다.
        # "--parallel", str(is_parallel),    # 💡 불리언도 str()로 감싸 'True'/'False' 문자열로 변환합니다. this makes result in correct
    ]

    print("🚀 실행할 명령어:")
    # 보기 좋게 한 줄씩 출력
    print(" \\\n    ".join(command_list))
    print("-" * 50)

    try:
        # command_list를 subprocess.run에 바로 전달
        result = subprocess.run(
            command_list, 
            check=True, 
            capture_output=True, 
            text=True,
            encoding='utf-8'
        )

        print("✅ 명령 실행 성공!")
        print("\n--- 표준 출력 (stdout) ---")
        print(result.stdout)
        
        if result.stderr:
            print("\n--- 표준 오류 (stderr) ---")
            print(result.stderr)

    except FileNotFoundError:
        print(f"❌ 오류: 'python' 또는 스크립트 파일을 찾을 수 없습니다. 경로를 확인해주세요.")

    except subprocess.CalledProcessError as e:
        print(f"❌ 명령 실행 실패! (Return Code: {e.returncode})")
        print("\n--- 표준 출력 (stdout) ---")
        print(e.stdout)
        print("\n--- 표준 오류 (stderr) ---")
        print(e.stderr)

# 함수 실행
if __name__ == "__main__":
    run_preprocessing()

🚀 실행할 명령어:
python \
    preprocess_and_generate_answer.py \
    --input_dir \
    /workspace/results/retrieval_docs/250912_182734/ \
    --max_rank \
    5 \
    --parallel \
    True
--------------------------------------------------
✅ 명령 실행 성공!

--- 표준 출력 (stdout) ---
Starting parallel processing for 50 files with max 10 workers...
--- Processing file: row_000024_Alibaba-NLP-gte-multilingual-base_382ab648.json ---
--- Processing file: row_000021_Alibaba-NLP-gte-multilingual-base_8588f043.json ---
--- Processing file: row_000011_Alibaba-NLP-gte-multilingual-base_0cf06061.json ---
--- Processing file: row_000039_Alibaba-NLP-gte-multilingual-base_a698801e.json ---
--- Processing file: row_000033_Alibaba-NLP-gte-multilingual-base_820896ec.json ---
--- Processing file: row_000023_Alibaba-NLP-gte-multilingual-base_8b30d2d8.json ---
--- Processing file: row_000040_Alibaba-NLP-gte-multilingual-base_5dd0e73e.json ---
--- Processing file: row_000031_Alibaba-NLP-gte-multilingual-base_9c35c0ae.j

In [2]:
import os 
import datetime
from final_result import process_json_files, append_new_data_frame_to_base_csv
# Define the path to your JSON files.
# IMPORTANT: Replace this with the actual path to your files.
# For example: '/workspace/results/final_result/'
# 현재 날짜와 시간을 가져옵니다.
now = datetime.datetime.now()

# 원하는 형식(yymmdd_hhmmss)으로 문자열을 만듭니다.
timestamp_str = now.strftime("%y%m%d_%H%M%S")
retrival_docs_folder = "/workspace/data/expr/final_result"
output_directory = (
    "/workspace/data/expr/competition_submission/" + timestamp_str + "/"
)  # Assuming the files are in the same directory as the script for this example

# Define the desired path for the output CSV file.
output_file = "final_predictions_" + timestamp_str + ".csv"
base_csv_uri = "/workspace/data/rag_test_data/scion.csv"
# Create dummy files for demonstration purposes
# In your real case, you would already have these files.
if not os.path.exists(output_directory):
    os.makedirs(output_directory)

new_df = process_json_files(retrival_docs_folder)
print(new_df)
append_new_data_frame_to_base_csv(
    base_csv_uri, output_directory, output_file, new_df
)


            id Prediction_retrieved_article_name_1  \
0   row_000001                                  없음   
1   row_000003                                  없음   
2   row_000004                                  없음   
3   row_000005                                  없음   
4   row_000006                                  없음   
5   row_000007                                  없음   
6   row_000011                                  없음   
7   row_000012                                  없음   
8   row_000013                                  없음   
9   row_000014                                  없음   
10  row_000015                                  없음   
11  row_000016                                  없음   
12  row_000017                                  없음   
13  row_000018                                  없음   
14  row_000019                                  없음   
15  row_000020                                  없음   
16  row_000021                                  없음   
17  row_000022              

configure base execution
python -m retrieval_system.main   --config_json /workspace/configs/query_encoder/config_PwC-Embedding_expr.json   --questions_jsonl /workspace/data/expr/singlehop_decompose.jsonl   --range 1-50   --top_k 50   --device auto --schema_json /workspace/configs/csv_schema/test_3.json

excution by manual
python -m retrieval_system.main   --vectordb_csv /workspace/results/vectordb/250911_072731/vector_db_PwC_test_SamilPwC-AXNode-GenAI_PwC-Embedding_expr.csv   --config_json /workspace/configs/query_encoder/config_gte-multilingual-base.json   --questions_jsonl /workspace/data/expr/singlehop_decompose.jsonl   --range 1-50   --top_k 50   --device auto --schema_json /workspace/configs/csv_schema/test_3.json